In [66]:
import tensorflow as tf
tf.__version__

'1.10.0'

In [67]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y+y+2

#### 執行會話

In [68]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

sess.close()

42


#### 使用 with

In [69]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


#### 使用global initializer

In [70]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


### Graph

In [71]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [72]:
x2 = tf.Variable(2)
x2.graph is tf.get_default_graph() and x1.graph is tf.get_default_graph()

True

In [73]:
# 創建新的圖，在with中設為默認圖
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [74]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
m,n

(20640, 8)

In [75]:
bias = np.ones((m,1))
bias

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [76]:
housing.data

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [77]:
housing.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [78]:
# 加上 bias
housing_data_plus_bias = np.c_[bias, housing.data]

#### 標準方程

In [79]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
XT

<tf.Tensor 'transpose:0' shape=(9, 20640) dtype=float32>

In [80]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [81]:
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


#### 梯度下降

In [82]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n+1,1],-1,1), name="theta")
y_pred = tf.matmul(X, theta,name="prediction") # 矩陣相乘

n_epochs = 1000
learning_rate = 0.01
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m*tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
           print("Epoch", epoch, ",MSE=",mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 ,MSE= 7.9712324
Epoch 100 ,MSE= 0.7275899
Epoch 200 ,MSE= 0.57162714
Epoch 300 ,MSE= 0.5548316
Epoch 400 ,MSE= 0.54622567
Epoch 500 ,MSE= 0.5401933
Epoch 600 ,MSE= 0.5358404
Epoch 700 ,MSE= 0.53268963
Epoch 800 ,MSE= 0.5304072
Epoch 900 ,MSE= 0.5287523


#### Tensorflow 提供許多優化器

In [83]:
# GradientDescentOptimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
           print("Epoch", epoch, ",MSE=",mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 ,MSE= 14.344283
Epoch 100 ,MSE= 0.827781
Epoch 200 ,MSE= 0.59213203
Epoch 300 ,MSE= 0.57323176
Epoch 400 ,MSE= 0.56174684
Epoch 500 ,MSE= 0.5531428
Epoch 600 ,MSE= 0.54663277
Epoch 700 ,MSE= 0.54168344
Epoch 800 ,MSE= 0.53790134
Epoch 900 ,MSE= 0.5349971


In [84]:
# MomentumOptimizer
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
           print("Epoch", epoch, ",MSE=",mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 ,MSE= 5.7922955
Epoch 100 ,MSE= 0.5339432
Epoch 200 ,MSE= 0.5248384
Epoch 300 ,MSE= 0.5243767
Epoch 400 ,MSE= 0.52432835
Epoch 500 ,MSE= 0.52432173
Epoch 600 ,MSE= 0.524321
Epoch 700 ,MSE= 0.52432096
Epoch 800 ,MSE= 0.52432084
Epoch 900 ,MSE= 0.5243208


### Placeholder

In [85]:
A = tf.placeholder(tf.float32, shape=(None,3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [86]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X") # n + 1 (bias)
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_epochs = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(batch_index, batch_size):
    start = batch_index*batch_size
    end = start + batch_size
    bias = np.ones((m,1))
    X_batch = np.c_[bias, housing.data][start:end]
    y_batch = housing.target.reshape(-1,1)[start:end]
    return X_batch, y_batch
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
        if epoch%10==0: 
            print("Epoch", epoch, ",MSE=",mse.eval())
        best_theta = theta.eval()

Epoch 0 ,MSE= 0.52461725
Epoch 10 ,MSE= 0.52432096
Epoch 20 ,MSE= 0.52432096
Epoch 30 ,MSE= 0.52432096
Epoch 40 ,MSE= 0.52432096
Epoch 50 ,MSE= 0.52432096
Epoch 60 ,MSE= 0.52432096
Epoch 70 ,MSE= 0.52432096
Epoch 80 ,MSE= 0.52432096
Epoch 90 ,MSE= 0.52432096


### Save the Model

In [87]:
# GradientDescentOptimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

n_epochs = 1000
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch", epoch, ",MSE=",mse.eval())
            save_path = saver.save(sess,"./tmp/my_model.ckpt")
        sess.run(training_op)
        
    best_theta = theta.eval()
    saver_path = saver.save(sess,"./tmp/my_model_final.ckpt")

Epoch 0 ,MSE= 16.312407
Epoch 100 ,MSE= 0.7877982
Epoch 200 ,MSE= 0.5685022
Epoch 300 ,MSE= 0.55353594
Epoch 400 ,MSE= 0.5453354
Epoch 500 ,MSE= 0.5394775
Epoch 600 ,MSE= 0.53525424
Epoch 700 ,MSE= 0.5322081
Epoch 800 ,MSE= 0.5300103
Epoch 900 ,MSE= 0.52842546


In [88]:
with tf.Session() as sess:
    saver.restore(sess,"./tmp/my_model_final.ckpt")
    print("mse:",mse.eval())
    sess.run(training_op)
    print("mes:",mse.eval())

INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt
mse: 0.52728176
mes: 0.5272722


### Tensorboard

In [89]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")

root_logdir = "./tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch", epoch, ",MSE=",mse.eval())
            summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
            file_writer.add_summary(summary_str, epoch)
        sess.run(training_op)
        
file_writer.close() #記得關閉

Epoch 0 ,MSE= 6.1779857
Epoch 100 ,MSE= 0.7257416
Epoch 200 ,MSE= 0.6198346
Epoch 300 ,MSE= 0.5927179
Epoch 400 ,MSE= 0.573918
Epoch 500 ,MSE= 0.560328
Epoch 600 ,MSE= 0.55048704
Epoch 700 ,MSE= 0.54335636
Epoch 800 ,MSE= 0.53818625
Epoch 900 ,MSE= 0.534434


運行 tensorboard --logdir tf_logs/

### Name Scope

In [90]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    
print(error.op.name)
print(mse.op.name)

loss/sub
loss/mse


In [91]:
def relu(X):
    with tf.name_scope("relu"): # 放進域名
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weight") # Outputs random values from a normal distribution
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")
tf.reset_default_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(output, feed_dict={X:[[1,2,3],[3,9,1]]})
    with tf.summary.FileWriter('./graph', sess.graph) as train_writer:
      train_writer.close()

### Share Variables

In [97]:
# 使用 def 參數
def relu(X, threshold): # ----
    with tf.name_scope("relu"): 
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weight") # Outputs random values from a normal distribution
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max") # ----

In [98]:
# 使用 scope
# with tf.variable_scope("relu"): # 創建 relu/threshold 變數
#     threshold = tf.get_variable("threshold", shape=(),
#                                 initializer=tf.constant_initializer(0.0)) 

# reuse : 復用變數， 如果變數存在的話使用既有的變數
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold") # 獲取既有的變量
    
with tf.variable_scope("relu") as scope: 
    scope.reuse_variables()# 復用變數
    threshold = tf.get_variable("threshold")